In [88]:
!pip install neo4j

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [89]:
from neo4j import GraphDatabase

# Set the connection details
uri = "neo4j://localhost:7687"
username = "neo4j"
password = "passcode"

# Create the driver and session objects
driver = GraphDatabase.driver(uri, auth=(username, password),encrypted=False)
session = driver.session()
# Run a query
result = session.run("MATCH (n) RETURN COUNT(n)")
# Print the result
for record in result:
    print(record[0])

24679


In [90]:
import pandas as pd
movies = pd.read_csv("movies_data.csv")
movies.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,#Gadhvi (He thought he was Gandhi),2019,109,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
1,#Yaaram,2019,110,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
2,...Aur Pyaar Ho Gaya,1997,147,"Comedy, Drama, Musical",4.7,827,Rahul Rawail,Bobby Deol,Aishwarya Rai Bachchan,Shammi Kapoor
3,...Yahaan,2005,142,"Drama, Romance, War",7.4,1086,Shoojit Sircar,Jimmy Sheirgill,Minissha Lamba,Yashpal Sharma
4,?: A Question Mark,2012,82,"Horror, Mystery, Thriller",5.6,326,Allyson Patel,Yash Dave,Muntazir Ahmad,Kiran Bhatia


In [82]:
for movie in range(len(movies)):
    query = "CREATE (:Movie {Name: $Name, Year: $Year, Rating: $Rating})"
    params = {'Name': movies.Name[movie], "Year": movies.Year[movie], "Rating": movies.Rating[movie]}
    with driver.session() as session:
        session.run(query, params)

In [83]:
# Create nodes for actors
for actor in range(len(movies)):
    query = "CREATE (:Actor {Actor1: $Actor1, Actor2: $Actor2, Actor3: $Actor3, Director: $Director})"
    params = {"Actor1": movies['Actor 1'][actor], "Actor2": movies['Actor 2'][actor], "Actor3": movies['Actor 3'][actor], "Director": movies['Director'][actor]}
    with driver.session() as session:
        session.run(query, params)

In [84]:
# Create relationships for actors who acted in movies
for movie_index, movie in movies.iterrows():
    query = "MATCH (a:Actor {Actor1: $Actor1, Actor2: $Actor2, Actor3: $Actor3}), (m:Movie {Name: $Name}) CREATE (a)-[:ACTED_IN]->(m)"
    params = {'Name': movie['Name'], "Actor1": movie['Actor 1'], "Actor2": movie['Actor 2'], "Actor3": movie['Actor 3']}
    with driver.session() as session:
        session.run(query, params)

In [85]:
for movie in range(len(movies)):
    query = "MATCH (m:Movie {Name: $Name}) CREATE (:Genre {Name: $Genre})"
    genres = movies['Genre'][movie].split(',')
    for genre in genres:
        params = {'Name': movies.Name[movie], 'Genre': genre.strip()}
        with driver.session() as session:
            session.run(query, params)

In [86]:
# Create relationships between movies and genres
for movie in range(len(movies)):
    query = "MATCH (m:Movie {Name: $Name}), (g:Genre {Name: $Genre}) CREATE (m)-[:BELONGS_TO]->(g)"
    genres = movies['Genre'][movie].split(',')
    for genre in genres:
        params = {'Name': movies.Name[movie], 'Genre': genre.strip()}
        with driver.session() as session:
            session.run(query, params)

In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [94]:
def get_movies(tx):
    movies = []
    for record in tx.run("MATCH (m:Movie) RETURN m.Name AS Name, m.Rating AS Rating"):
        movies.append(record)
    return movies

In [93]:
with driver.session() as session:
    movies = session.read_transaction(get_movies)

/var/folders/x7/fc3vnzb90sdcq1ydx20l7_9h0000gn/T/ipykernel_69194/2561505493.py:2: DeprecationWarning: read_transaction has been renamed to execute_read
  movies = session.read_transaction(get_movies)


In [95]:
import pandas as pd
movie_data = pd.DataFrame(movies, columns=["Name", "Rating"])

In [87]:
# Close the session and driver
session.close()
driver.close()